<a href="https://colab.research.google.com/github/koya-jp/google-colab-kohya/blob/master/%E8%87%AA%E4%BD%9CLoRA%E3%82%92%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 自作LoRAを作成 { display-mode: "both" }

# 参考：
# https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/
# https://indoortimes.net/index.php/ai/stable_diffusion/lora_making/
# https://blog.14nigo.net/2023/03/trytocreatelora.html?m=1

from google.colab import drive
drive.mount('/content/drive')

!sudo apt autoremove >/dev/null 2>&1
!sudo apt-get -y update >/dev/null 2>&1
!sudo apt-get -y install python3.10 >/dev/null 2>&1

!sudo cp `which python3.10` /usr/local/bin/python

!wget https://bootstrap.pypa.io/get-pip.py >/dev/null 2>&1
!sudo python get-pip.py >/dev/null 2>&1

!python --version
!pip --version

import os
os.makedirs("/content/drive/My Drive/work", exist_ok=True)
%cd '/content/drive/My Drive/work'

# パッケージのインストール
# git clone する
!git clone https://github.com/kohya-ss/sd-scripts >/dev/null 2>&1
%cd sd-scripts
print("current directory is -> ")
%pwd

!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117 >/dev/null 2>&1
!pip install --upgrade -r requirements.txt >/dev/null 2>&1
!pip install -U --pre triton >/dev/null 2>&1
!pip install xformers==0.0.16rc425 >/dev/null 2>&1

!accelerate config default --mixed_precision fp16 >/dev/null 2>&1

# sd-scriptsに配下に訓練用画像をコピーする
%cp -r /content/drive/MyDrive/Lora_rum_sama/Images /content/drive/My Drive/work/sd-scripts/

# 画像にキャプションを付与する
!python finetune/make_captions.py --batch_size 8 /content/drive/My Drive/work/sd-scripts/Images &> /dev/null
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 /content/drive/My Drive/work/sd-scripts/Images &> /dev/null

!python finetune/merge_captions_to_metadata.py /content/drive/My Drive/work/sd-scripts/Images meta_cap.json &> /dev/null
!python finetune/merge_dd_tags_to_metadata.py /content/drive/My Drive/work/sd-scripts/Images --in_json meta_cap.json meta_cap_dd.json &> /dev/null
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json &> /dev/null

!python finetune/prepare_buckets_latents.py /content/drive/My Drive/work/sd-scripts/Images meta_clean.json meta_lat.json /content/drive/MyDrive/Lora_rum_sama/Model/majicmixRealistic_v6.safetensors --batch_size 4 --max_resolution 512,512 --mixed_precision no &> /dev/null

!accelerate launch  --num_cpu_threads_per_process 1 train_network.py --pretrained_model_name_or_path=/content/drive/MyDrive/Lora_rum_sama/Model/majicmixRealistic_v6.safetensors --in_json meta_lat.json --train_data_dir=/content/drive/My Drive/work/sd-scripts/Images --output_dir=output_models --shuffle_caption --train_batch_size=1 --learning_rate=1e-4 --max_train_steps=400 --use_8bit_adam --xformers --gradient_checkpointing --mixed_precision=fp16 --save_every_n_epochs=4 --save_precision=fp16 --save_model_as=safetensors  --network_module=networks.lora

# 学習に使用した画像のプロンプトを確認する
!cat /content/drive/MyDrive/sd-scripts/Images/*.txt

# 最後の学習モデルをDriveに保存する
!cp output_models/last.safetensors /content/drive/MyDrive/Lora/pretty-cut-rum-sama.safetensors


Mounted at /content/drive
Python 3.10.12
pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
/content/drive/My Drive/work
/content/drive/My Drive/work/sd-scripts
current directory is -> 


In [ ]:
# 学習に使用した画像のプロンプトを確認する
!cat /content/drive/MyDrive/sd-scripts/Images/*.txt


In [ ]:
# google.colabライブラリのインポート
import google.colab

# ランタイムの接続を解除して削除
google.colab.runtime.unassign()